In [1]:
# Importar librerías.

import pandas as pd
import numpy as np
import geopandas
import rasterio
from shapely.geometry import Point, LineString

In [2]:
# Ubicación de shapes

# Cuencas
nombre_objeto_cuencas = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_CUENCAS_3857_20230104_NB.shp'

# Trayectorias hidráulicas
nombre_objeto_th = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_TRAYECTORIA_3857_20230104_NB.shp'

# Sumideros
nombre_objeto_sumideros = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_SUMIDEROS_3857_20230104_NB.shp'

# Nexos
nombre_objeto_nexos = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_NEXOS_3857_20230104_NB.shp'

# BR
nombre_objeto_br = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_BR_3857_20230104_NB.shp'

# Conductos originales
nombre_objeto_conductos = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_CONDUCTOS_3857_20230104_NB.shp'

# DEM en formato raster, se debe importar en el mismo SRC con el que se va a trabajar, deuda pendiente
nombre_objeto_dem = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\RASTER\DEM_5348.tif'

# Cuencas salida
nombre_objeto_cuencas_s = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_CUENCAS_3857_20230104_S.shp'

# Trayectorias hidráulicas salida
nombre_objeto_th_s = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_TRAYECTORIA_3857_20230104_S.shp'

# Sumideros salida
nombre_objeto_sumideros_s = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_SUMIDEROS_3857_20230104_S.shp'

# Nexos salida
nombre_objeto_nexos_s = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_NEXOS_3857_20230104_S.shp'

# BR salida
nombre_objeto_br_s = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_BR_3857_20230104_S.shp'

# Conductos originales salida
nombre_objeto_conductos_s = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\O2023B_CONDUCTOS_3857_20230104_S.shp'

In [3]:
# Sistema de coordenadas de referencia a partir de su número EPSG.

epsg = 5348

In [4]:
# Importar shape de cuencas y cálculo de áreas [ha].

cuencas = geopandas.read_file(nombre_objeto_cuencas)
cuencas.to_crs(crs = epsg, inplace = True)
cuencas['id_cuenca'] = cuencas.index
cuencas = cuencas[['id_cuenca','geometry']]
cuencas['area'] = cuencas.area.round(0)/10000
cuencas.head()

,id_cuenca,geometry,area
0,0,"POLYGON ((6365287.852 6173601.143, 6365406.591...",3.0652
1,1,"POLYGON ((6365406.591 6173474.439, 6365466.067...",0.6379
2,2,"POLYGON ((6365406.591 6173474.439, 6365470.918...",1.3737
3,3,"POLYGON ((6365287.852 6173601.143, 6365357.205...",3.2173
4,4,"POLYGON ((6365446.554 6173713.904, 6365468.088...",0.8848


In [5]:
# Importar DEM

dem = rasterio.open(nombre_objeto_dem)

In [6]:
# Importar shape de sumideros

sumideros = geopandas.read_file(nombre_objeto_sumideros)
sumideros.to_crs(crs = epsg, inplace = True)
sumideros['id_sumidero'] = sumideros.index
sumideros = sumideros[['id_sumidero','geometry']]
sumideros.head()

,id_sumidero,geometry
0,0,POINT (6365478.749 6173391.814)
1,1,POINT (6365488.387 6173360.820)
2,2,POINT (6365508.486 6173363.253)
3,3,POINT (6365580.751 6173427.184)
4,4,POINT (6365554.901 6173435.510)


In [7]:
# Obtener coordenadas CTN de sumideros

sumideros['CTN'] = [round(float(i),2) for i in 
                  dem.sample([(x,y) for x,y in zip(sumideros['geometry'].x , sumideros['geometry'].y)])]
sumideros.head()

,id_sumidero,geometry,CTN
0,0,POINT (6365478.749 6173391.814),19.83
1,1,POINT (6365488.387 6173360.820),19.90
2,2,POINT (6365508.486 6173363.253),19.66
3,3,POINT (6365580.751 6173427.184),19.00
4,4,POINT (6365554.901 6173435.510),19.25


In [8]:
# Importar trayectorias hidráulicas y cálculo de longitudes

th = geopandas.read_file(nombre_objeto_th)
th.to_crs(crs = epsg, inplace = True)
th['id_th'] = th.index
th = th[['id_th', 'geometry']]
th['longitud'] = th.length.round(1)
th.head()

,id_th,geometry,longitud
0,0,"LINESTRING (6365291.198 6173592.555, 6365404.2...",274.7
1,1,"LINESTRING (6365389.711 6173290.575, 6365488.3...",121.1
2,2,"LINESTRING (6365553.249 6173265.368, 6365508.4...",107.6
3,3,"LINESTRING (6365513.978 6173380.163, 6365580.7...",81.7
4,4,"LINESTRING (6365499.892 6173396.103, 6365554.9...",67.7


In [9]:
# Obtención de cotas de la trayectoria hidráulica y cálculo de pendiente

th['CTNi'] = [round(float(i),2) for i in dem.sample([(x,y) for x,y in zip(
    geopandas.GeoDataFrame(crs=th.crs, geometry=list(Point(th.geometry.iloc[i].coords[0]) 
                                                     for i in range(len(th))))['geometry'].x , 
    geopandas.GeoDataFrame(crs=th.crs, geometry=list(Point(th.geometry.iloc[i].coords[0]) 
                                                     for i in range(len(th))))['geometry'].y)])]
th['CTNf'] = [round(float(i),2) for i in dem.sample([(x,y) for x,y in zip(
    geopandas.GeoDataFrame(crs=th.crs, geometry=list(Point(th.geometry.iloc[i].coords[1]) 
                                                     for i in range(len(th))))['geometry'].x , 
    geopandas.GeoDataFrame(crs=th.crs, geometry=list(Point(th.geometry.iloc[i].coords[1]) 
                                                     for i in range(len(th))))['geometry'].y)])]
th['i_TN'] = (th['CTNi']-th['CTNf'])/th['longitud']

# Nota, por cuestión de urgencia se toma el valor absoluto de la pendiente sin tener en cuenta posibles errores en el trazado.
# Se deben independizar los errores a futuro obteniendo el nodo final de la th a partir de la union espacial con los sumideros.

th['i_adop'] = [0.0005 if abs(th['i_TN'].iloc[i]) <= 0.0005 else abs(th['i_TN'].iloc[i]) for i in range(len(th))]
th.head()

,id_th,geometry,longitud,CTNi,CTNf,i_TN,i_adop
0,0,"LINESTRING (6365291.198 6173592.555, 6365404.2...",274.7,20.00,20.00,0.000000,0.000500
1,1,"LINESTRING (6365389.711 6173290.575, 6365488.3...",121.1,20.00,19.90,0.000826,0.000826
2,2,"LINESTRING (6365553.249 6173265.368, 6365508.4...",107.6,19.89,19.66,0.002138,0.002138
3,3,"LINESTRING (6365513.978 6173380.163, 6365580.7...",81.7,19.50,19.00,0.006120,0.006120
4,4,"LINESTRING (6365499.892 6173396.103, 6365554.9...",67.7,19.66,19.25,0.006056,0.006056


In [10]:
# Cálculo del tiempo de concentración, establecer el tiempo de mojado "t_mojado" en minutos.

t_mojado = 5
th['tc'] = [t_mojado + round(th['longitud'].iloc[i]/(8.8*60*th['i_adop'].iloc[i]**0.5),1) for i in range(len(th))]
th.head()

,id_th,geometry,longitud,CTNi,CTNf,i_TN,i_adop,tc
0,0,"LINESTRING (6365291.198 6173592.555, 6365404.2...",274.7,20.00,20.00,0.000000,0.000500,28.3
1,1,"LINESTRING (6365389.711 6173290.575, 6365488.3...",121.1,20.00,19.90,0.000826,0.000826,13.0
2,2,"LINESTRING (6365553.249 6173265.368, 6365508.4...",107.6,19.89,19.66,0.002138,0.002138,9.4
3,3,"LINESTRING (6365513.978 6173380.163, 6365580.7...",81.7,19.50,19.00,0.006120,0.006120,7.0
4,4,"LINESTRING (6365499.892 6173396.103, 6365554.9...",67.7,19.66,19.25,0.006056,0.006056,6.6


In [11]:
# Uniones espaciales entre sumideros, cuencas y trayectorias hidráulicas.

sumideros_m = sumideros.sjoin_nearest(th, how='left')
sumideros_m.drop(['index_right'], axis=1, inplace=True)
sumideros_m = sumideros_m.sjoin(cuencas, how='left')
sumideros_m.drop(['longitud','CTNi','CTNf','i_TN','i_adop','index_right'], axis=1, inplace=True)
sumideros_m.head()

,id_sumidero,geometry,CTN,id_th,tc,id_cuenca,area
0,0,POINT (6365478.749 6173391.814),19.83,0,28.3,0,3.0652
1,1,POINT (6365488.387 6173360.820),19.90,1,13.0,5,0.5874
2,2,POINT (6365508.486 6173363.253),19.66,2,9.4,6,0.9132
3,3,POINT (6365580.751 6173427.184),19.00,3,7.0,7,0.3894
4,4,POINT (6365554.901 6173435.510),19.25,4,6.6,1,0.6379


In [12]:
# Dimensionado de sumideros, caudales en m³/s

C = 0.8
sumideros_m['I_10'] = [int(584.43*sumideros_m['tc'].iloc[i]**-.611) for i in range(len(sumideros_m))]
sumideros_m['Q_10'] = round(C * sumideros_m['I_10'] * sumideros_m['area'] / 360, 2)
sumideros_m['I_5'] = [int(514.81*sumideros_m['tc'].iloc[i]**-.611) for i in range(len(sumideros_m))]
sumideros_m['Q_5'] = round(C * sumideros_m['I_5'] * sumideros_m['area'] / 360, 2)
sumideros_m['I_2'] = [int(435.3*sumideros_m['tc'].iloc[i]**-.611) for i in range(len(sumideros_m))]
sumideros_m['Q_2'] = round(C * sumideros_m['I_2'] * sumideros_m['area'] / 360, 2)

# Capacidad de sumideros 100 l/m (1 cuerpo)

cap_sum = 100

sumideros_m['n_cuerpos'] = round(sumideros_m['Q_10']/cap_sum*1000,0)

sumideros_m.head()

,id_sumidero,geometry,CTN,id_th,tc,id_cuenca,area,I_10,Q_10,I_5,Q_5,I_2,Q_2,n_cuerpos
0,0,POINT (6365478.749 6173391.814),19.83,0,28.3,0,3.0652,75,0.51,66,0.45,56,0.38,5.0
1,1,POINT (6365488.387 6173360.820),19.90,1,13.0,5,0.5874,121,0.16,107,0.14,90,0.12,2.0
2,2,POINT (6365508.486 6173363.253),19.66,2,9.4,6,0.9132,148,0.30,130,0.26,110,0.22,3.0
3,3,POINT (6365580.751 6173427.184),19.00,3,7.0,7,0.3894,177,0.15,156,0.13,132,0.11,2.0
4,4,POINT (6365554.901 6173435.510),19.25,4,6.6,1,0.6379,184,0.26,162,0.23,137,0.19,3.0


In [13]:
# Importar nexos

nexos = geopandas.read_file(nombre_objeto_nexos)
nexos.to_crs(crs = epsg, inplace = True)
nexos['id_nexo'] = nexos.index
nexos = nexos[['id_nexo', 'geometry']]
nexos['longitud'] = nexos.length.round(1)
nexos.head()

,id_nexo,geometry,longitud
0,0,"LINESTRING (6365478.749 6173391.814, 6365495.4...",19.7
1,1,"LINESTRING (6365488.387 6173360.820, 6365495.4...",21.7
2,2,"LINESTRING (6365508.486 6173363.253, 6365495.4...",22.3
3,3,"LINESTRING (6365580.751 6173427.184, 6365576.2...",11.4
4,4,"LINESTRING (6365554.901 6173435.510, 6365576.2...",21.5


In [14]:
# Determinar cotas y pendiente del nexo. NOTA: nuevamente se adopta valor absoluto por disponibilidad de tiempo

nexos['CTNi'] = [round(float(i),2) for i in dem.sample([(x,y) for x,y in zip(
    geopandas.GeoDataFrame(crs=nexos.crs, geometry=list(Point(nexos.geometry.iloc[i].coords[0]) 
                                                     for i in range(len(nexos))))['geometry'].x , 
    geopandas.GeoDataFrame(crs=nexos.crs, geometry=list(Point(nexos.geometry.iloc[i].coords[0]) 
                                                     for i in range(len(nexos))))['geometry'].y)])]
nexos['CTNf'] = [round(float(i),2) for i in dem.sample([(x,y) for x,y in zip(
    geopandas.GeoDataFrame(crs=nexos.crs, geometry=list(Point(nexos.geometry.iloc[i].coords[1]) 
                                                     for i in range(len(nexos))))['geometry'].x , 
    geopandas.GeoDataFrame(crs=nexos.crs, geometry=list(Point(nexos.geometry.iloc[i].coords[1]) 
                                                     for i in range(len(nexos))))['geometry'].y)])]
tapada_i = 0.8
tapada_f = 1.2

nexos['i'] = abs((nexos['CTNi']-tapada_i)-(nexos['CTNf']-tapada_f))/nexos['longitud']
nexos.head()

,id_nexo,geometry,longitud,CTNi,CTNf,i
0,0,"LINESTRING (6365478.749 6173391.814, 6365495.4...",19.7,19.83,19.68,0.027919
1,1,"LINESTRING (6365488.387 6173360.820, 6365495.4...",21.7,19.90,19.68,0.028571
2,2,"LINESTRING (6365508.486 6173363.253, 6365495.4...",22.3,19.66,19.68,0.017040
3,3,"LINESTRING (6365580.751 6173427.184, 6365576.2...",11.4,19.00,19.06,0.029825
4,4,"LINESTRING (6365554.901 6173435.510, 6365576.2...",21.5,19.25,19.06,0.027442


In [15]:
# Unión espacial de nexos y sumideros

nexos_m = nexos.sjoin_nearest(sumideros_m, how='left')
nexos_m.drop(['index_right', 'n_cuerpos', 'I_10', 'I_5', 'I_2','CTN'], axis=1, inplace=True)
nexos_m.head()

,id_nexo,geometry,longitud,CTNi,CTNf,i,id_sumidero,id_th,tc,id_cuenca,area,Q_10,Q_5,Q_2
0,0,"LINESTRING (6365478.749 6173391.814, 6365495.4...",19.7,19.83,19.68,0.027919,0,0,28.3,0,3.0652,0.51,0.45,0.38
1,1,"LINESTRING (6365488.387 6173360.820, 6365495.4...",21.7,19.90,19.68,0.028571,1,1,13.0,5,0.5874,0.16,0.14,0.12
2,2,"LINESTRING (6365508.486 6173363.253, 6365495.4...",22.3,19.66,19.68,0.017040,2,2,9.4,6,0.9132,0.30,0.26,0.22
3,3,"LINESTRING (6365580.751 6173427.184, 6365576.2...",11.4,19.00,19.06,0.029825,3,3,7.0,7,0.3894,0.15,0.13,0.11
4,4,"LINESTRING (6365554.901 6173435.510, 6365576.2...",21.5,19.25,19.06,0.027442,4,4,6.6,1,0.6379,0.26,0.23,0.19


In [16]:
# Dimensionado de nexos

n = 0.012
nexos_m['D_10'] = round((nexos_m['Q_10']*n/nexos_m['i']**0.5*4**(5/3)/np.pi)**(3/8),2)
nexos_m['D_5'] = round((nexos_m['Q_5']*n/nexos_m['i']**0.5*4**(5/3)/np.pi)**(3/8),2)
nexos_m['D_2'] = round((nexos_m['Q_2']*n/nexos_m['i']**0.5*4**(5/3)/np.pi)**(3/8),2)
nexos_m['D'] = [round(nexos_m['D_10'].iloc[i],1)*1000 if round(nexos_m['D_10'].iloc[i],1)*1000>=400 else 400 for i in range(len(nexos_m))]
nexos_m['V'] = round(nexos_m['Q_10']/(np.pi/4*(nexos_m['D']/1000)**2),1)
nexos_m['tv_tc'] = round(nexos_m['longitud']/nexos_m['V']/60,1) + nexos_m['tc']
nexos_m.head()

,id_nexo,geometry,longitud,CTNi,CTNf,i,id_sumidero,id_th,tc,id_cuenca,area,Q_10,Q_5,Q_2,D_10,D_5,D_2,D,V,tv_tc
0,0,"LINESTRING (6365478.749 6173391.814, 6365495.4...",19.7,19.83,19.68,0.027919,0,0,28.3,0,3.0652,0.51,0.45,0.38,0.45,0.43,0.40,400.0,4.1,28.4
1,1,"LINESTRING (6365488.387 6173360.820, 6365495.4...",21.7,19.90,19.68,0.028571,1,1,13.0,5,0.5874,0.16,0.14,0.12,0.29,0.27,0.26,400.0,1.3,13.3
2,2,"LINESTRING (6365508.486 6173363.253, 6365495.4...",22.3,19.66,19.68,0.017040,2,2,9.4,6,0.9132,0.30,0.26,0.22,0.40,0.38,0.36,400.0,2.4,9.6
3,3,"LINESTRING (6365580.751 6173427.184, 6365576.2...",11.4,19.00,19.06,0.029825,3,3,7.0,7,0.3894,0.15,0.13,0.11,0.28,0.27,0.25,400.0,1.2,7.2
4,4,"LINESTRING (6365554.901 6173435.510, 6365576.2...",21.5,19.25,19.06,0.027442,4,4,6.6,1,0.6379,0.26,0.23,0.19,0.35,0.33,0.31,400.0,2.1,6.8


In [17]:
# Importar conductos

conductos = geopandas.read_file(nombre_objeto_conductos)
conductos.to_crs(crs = epsg, inplace = True)
conductos['id_conducto'] = conductos.index
conductos = conductos[['id_conducto','geometry']]
conductos['longitud'] = conductos.length.round(1)
conductos.head()

,id_conducto,geometry,longitud
0,0,"LINESTRING (6365495.474 6173381.331, 6365576.2...",98.6
1,1,"LINESTRING (6365576.298 6173437.724, 6365647.3...",86.4
2,2,"LINESTRING (6365647.387 6173486.807, 6365727.4...",97.9
3,3,"LINESTRING (6360984.651 6169800.629, 6360993.1...",14.0
4,4,"LINESTRING (6360993.137 6169789.552, 6361069.9...",122.6


In [18]:
# Obtención de cotas del TN de conductos. Por cuestion de tiempo no se invierten tramos automaticamente

conductos['CTNi'] = [round(float(i),2) for i in dem.sample([(x,y) for x,y in zip(
    geopandas.GeoDataFrame(crs=conductos.crs, geometry=list(Point(conductos.geometry.iloc[i].coords[0]) 
                                                     for i in range(len(conductos))))['geometry'].x , 
    geopandas.GeoDataFrame(crs=conductos.crs, geometry=list(Point(conductos.geometry.iloc[i].coords[0]) 
                                                     for i in range(len(conductos))))['geometry'].y)])]
conductos['CTNf'] = [round(float(i),2) for i in dem.sample([(x,y) for x,y in zip(
    geopandas.GeoDataFrame(crs=conductos.crs, geometry=list(Point(conductos.geometry.iloc[i].coords[1]) 
                                                     for i in range(len(conductos))))['geometry'].x , 
    geopandas.GeoDataFrame(crs=conductos.crs, geometry=list(Point(conductos.geometry.iloc[i].coords[1]) 
                                                     for i in range(len(conductos))))['geometry'].y)])]
conductos['i'] = (conductos['CTNi']-conductos['CTNf'])/conductos['longitud']
conductos['i_conducto'] = [max(conductos['i'][j],0.003) for j in range(len(conductos))]
conductos.head()

,id_conducto,geometry,longitud,CTNi,CTNf,i,i_conducto
0,0,"LINESTRING (6365495.474 6173381.331, 6365576.2...",98.6,19.68,19.06,0.006288,0.006288
1,1,"LINESTRING (6365576.298 6173437.724, 6365647.3...",86.4,19.06,17.86,0.013889,0.013889
2,2,"LINESTRING (6365647.387 6173486.807, 6365727.4...",97.9,17.86,16.75,0.011338,0.011338
3,3,"LINESTRING (6360984.651 6169800.629, 6360993.1...",14.0,26.00,25.96,0.002857,0.003000
4,4,"LINESTRING (6360993.137 6169789.552, 6361069.9...",122.6,25.96,25.61,0.002855,0.003000


In [19]:
# Conductos con pendiente negativa del TN

conductos.loc[conductos['i'] < 0]

,id_conducto,geometry,longitud,CTNi,CTNf,i,i_conducto


In [20]:
# Importación de BR

br = geopandas.read_file(nombre_objeto_br)
br.to_crs(crs = epsg, inplace = True)
br['id_br'] = br.index
br = br[['id_br','geometry']]
br.head()

,id_br,geometry
0,0,POINT (6365495.474 6173381.331)
1,1,POINT (6365576.298 6173437.724)
2,2,POINT (6365647.387 6173486.807)
3,3,POINT (6360984.651 6169800.629)
4,4,POINT (6360993.137 6169789.552)


In [21]:
# Unión espacial de nexos con BR

nexos_br = nexos_m[['area','geometry','tv_tc']].sjoin_nearest(br, how='left')
nexos_br.head()

,area,geometry,tv_tc,index_right,id_br
0,3.0652,"LINESTRING (6365478.749 6173391.814, 6365495.4...",28.4,0,0
1,0.5874,"LINESTRING (6365488.387 6173360.820, 6365495.4...",13.3,0,0
2,0.9132,"LINESTRING (6365508.486 6173363.253, 6365495.4...",9.6,0,0
3,0.3894,"LINESTRING (6365580.751 6173427.184, 6365576.2...",7.2,1,1
4,0.6379,"LINESTRING (6365554.901 6173435.510, 6365576.2...",6.8,1,1


In [22]:
br['area_br'] = [sum(nexos_br['area'].loc[nexos_br['id_br'] == br['id_br'][i]]) for i in range(len(br))]
br.head()

,id_br,geometry,area_br
0,0,POINT (6365495.474 6173381.331),4.5658
1,1,POINT (6365576.298 6173437.724),1.0273
2,2,POINT (6365647.387 6173486.807),5.4758
3,3,POINT (6360984.651 6169800.629),2.1891
4,4,POINT (6360993.137 6169789.552),2.0797


In [23]:
# Tiempos de concentración para cada BR

br['tv_tc'] = [max(nexos_br['tv_tc'].loc[nexos_br['id_br'] == br['id_br'][i]]) if 
               sum(nexos_br['tv_tc'].loc[nexos_br['id_br'] == br['id_br'][i]])>0  else 0 for i in range(len(br))]
br.head()

,id_br,geometry,area_br,tv_tc
0,0,POINT (6365495.474 6173381.331),4.5658,28.4
1,1,POINT (6365576.298 6173437.724),1.0273,7.2
2,2,POINT (6365647.387 6173486.807),5.4758,46.0
3,3,POINT (6360984.651 6169800.629),2.1891,27.7
4,4,POINT (6360993.137 6169789.552),2.0797,27.5


In [24]:
# Puntos iniciales y puntos finales de conducto

puntos_iniciales = geopandas.GeoDataFrame(crs=conductos.crs, 
                                          geometry=list(Point(conductos.geometry.iloc[i].coords[0]) for i in range(len(conductos))))
puntos_finales = geopandas.GeoDataFrame(crs=conductos.crs, 
                                        geometry=list(Point(conductos.geometry.iloc[i].coords[1]) for i in range(len(conductos))))
puntos_iniciales.head()

,geometry
0,POINT (6365495.474 6173381.331)
1,POINT (6365576.298 6173437.724)
2,POINT (6365647.387 6173486.807)
3,POINT (6360984.651 6169800.629)
4,POINT (6360993.137 6169789.552)


In [25]:
# Identificación con BR

conductos['br_inicial'] = puntos_iniciales.sjoin_nearest(br, how='left')['id_br']
conductos['area_br'] = puntos_iniciales.sjoin_nearest(br, how='left')['area_br']
conductos['tv_tc'] = puntos_iniciales.sjoin_nearest(br, how='left')['tv_tc']
conductos['br_final'] = puntos_finales.sjoin_nearest(br, how='left')['id_br']
conductos.head()

,id_conducto,geometry,longitud,CTNi,CTNf,i,i_conducto,br_inicial,area_br,tv_tc,br_final
0,0,"LINESTRING (6365495.474 6173381.331, 6365576.2...",98.6,19.68,19.06,0.006288,0.006288,0,4.5658,28.4,1
1,1,"LINESTRING (6365576.298 6173437.724, 6365647.3...",86.4,19.06,17.86,0.013889,0.013889,1,1.0273,7.2,2
2,2,"LINESTRING (6365647.387 6173486.807, 6365727.4...",97.9,17.86,16.75,0.011338,0.011338,2,5.4758,46.0,15
3,3,"LINESTRING (6360984.651 6169800.629, 6360993.1...",14.0,26.00,25.96,0.002857,0.003000,3,2.1891,27.7,4
4,4,"LINESTRING (6360993.137 6169789.552, 6361069.9...",122.6,25.96,25.61,0.002855,0.003000,4,2.0797,27.5,17


In [26]:
# Matriz de aportes

mat_apo = pd.DataFrame(index = [i for i in range(len(conductos))], columns = [i for i in range(len(conductos))], data = 0)
for i in range(len(conductos)):
    mat_apo[i] = conductos.index.isin(pd.Series(conductos.index).loc[conductos['br_final'] == conductos['br_inicial'][i]])*1
mat_apo.iloc[:7,:7]

,0,1,2,3,4,5,6
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0


In [27]:
# Por simplicidad se itera una cantidad de veces igual a la cantidad de tramos existentes.

k = 0
for k in range(len(conductos)):
    for i in range(len(conductos)):
        for j in range(len(conductos)):
            if mat_apo[i][j] == 1:
                mat_apo[i] = mat_apo[i] + mat_apo[j]
                mat_apo = (mat_apo/mat_apo).fillna(0)
mat_apo.iloc[:7,:7]       

,0,1,2,3,4,5,6
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# Se completa la matriz de aporte con el aporte propio de cada tramo

for i in range(len(conductos)):
    mat_apo[i][i] = 1
mat_apo.iloc[:7,:7]

,0,1,2,3,4,5,6
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [29]:
# Matriz de aporte tv_tc

mat_apo_tv_tc = mat_apo.copy()
for i in range(len(conductos)):
    for j in range(len(conductos)):
        mat_apo_tv_tc[i][j] = mat_apo[i][j]*conductos['tv_tc'][j]
mat_apo_tv_tc

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,28.4,28.4,28.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,7.2,7.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,27.7,27.7,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,27.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,30.8,30.8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,15.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.3,10.3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Matriz de aporte de áreas

mat_apo_area = mat_apo.copy()
for i in range(len(conductos)):
    for j in range(len(conductos)):
        mat_apo_area[i][j] = mat_apo[i][j]*conductos['area_br'][j]
mat_apo_area.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,4.5658,4.5658,4.5658,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0000,1.0273,1.0273,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0000,0.0000,5.4758,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0000,0.0000,0.0000,2.1891,2.1891,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0000,0.0000,0.0000,0.0000,2.0797,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# Cálculo del área acumulada

conductos['area_acum'] = [sum(mat_apo_area[i]) for i in range(len(conductos))]
conductos.head()

,id_conducto,geometry,longitud,CTNi,CTNf,i,i_conducto,br_inicial,area_br,tv_tc,br_final,area_acum
0,0,"LINESTRING (6365495.474 6173381.331, 6365576.2...",98.6,19.68,19.06,0.006288,0.006288,0,4.5658,28.4,1,4.5658
1,1,"LINESTRING (6365576.298 6173437.724, 6365647.3...",86.4,19.06,17.86,0.013889,0.013889,1,1.0273,7.2,2,5.5931
2,2,"LINESTRING (6365647.387 6173486.807, 6365727.4...",97.9,17.86,16.75,0.011338,0.011338,2,5.4758,46.0,15,11.0689
3,3,"LINESTRING (6360984.651 6169800.629, 6360993.1...",14.0,26.00,25.96,0.002857,0.003000,3,2.1891,27.7,4,2.1891
4,4,"LINESTRING (6360993.137 6169789.552, 6361069.9...",122.6,25.96,25.61,0.002855,0.003000,4,2.0797,27.5,17,4.2688


In [32]:
# Matriz para el cálculo de tiempos de viaje

mat_apo_0 = pd.DataFrame(index = [i for i in range(len(conductos))], columns = [i for i in range(len(conductos))], data = 0)
for i in range(len(conductos)):
    mat_apo_0[i] = conductos.index.isin(pd.Series(conductos.index).loc[conductos['br_final'] == conductos['br_inicial'][i]])*1
k = 0
for k in range(len(conductos)):
    for i in range(len(conductos)):
        for j in range(len(conductos)):
            if mat_apo_0[i][j] == 1:
                mat_apo_0[i] = mat_apo_0[i] + mat_apo_0[j]
                mat_apo_0 = (mat_apo_0/mat_apo_0).fillna(0)
                
conductos['tv'] = conductos['longitud']/1.5/60
tv_mat_apo = mat_apo_0*conductos['tv']
tv_mat_apo

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0,0.96,1.087778,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,0.0,0.00,1.087778,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
2,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,0.0,0.00,0.000000,0.0,1.362222,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
4,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
5,0.0,0.00,0.000000,0.0,0.000000,0.0,1.621111,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
6,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
7,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,2.261111,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
8,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
9,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


In [33]:
# Determinación del tiempo de concentración

tv = [sum([tv_mat_apo[i][j] for j in range(len(conductos))]) for i in range(len(conductos))]
tv_tc = [max([mat_apo_tv_tc[i][j] for j in range(len(conductos))]) for i in range(len(conductos))]
tv_tc
conductos['tc'] = [tv[i]+tv_tc[i] for i in range(len(conductos))]
conductos.head()

,id_conducto,geometry,longitud,CTNi,CTNf,i,i_conducto,br_inicial,area_br,tv_tc,br_final,area_acum,tv,tc
0,0,"LINESTRING (6365495.474 6173381.331, 6365576.2...",98.6,19.68,19.06,0.006288,0.006288,0,4.5658,28.4,1,4.5658,1.095556,28.400000
1,1,"LINESTRING (6365576.298 6173437.724, 6365647.3...",86.4,19.06,17.86,0.013889,0.013889,1,1.0273,7.2,2,5.5931,0.960000,29.360000
2,2,"LINESTRING (6365647.387 6173486.807, 6365727.4...",97.9,17.86,16.75,0.011338,0.011338,2,5.4758,46.0,15,11.0689,1.087778,48.175556
3,3,"LINESTRING (6360984.651 6169800.629, 6360993.1...",14.0,26.00,25.96,0.002857,0.003000,3,2.1891,27.7,4,2.1891,0.155556,27.700000
4,4,"LINESTRING (6360993.137 6169789.552, 6361069.9...",122.6,25.96,25.61,0.002855,0.003000,4,2.0797,27.5,17,4.2688,1.362222,29.062222


In [34]:
conductos

,id_conducto,geometry,longitud,CTNi,CTNf,i,i_conducto,br_inicial,area_br,tv_tc,br_final,area_acum,tv,tc
0,0,"LINESTRING (6365495.474 6173381.331, 6365576.2...",98.6,19.68,19.06,0.006288,0.006288,0,4.5658,28.4,1,4.5658,1.095556,28.400000
1,1,"LINESTRING (6365576.298 6173437.724, 6365647.3...",86.4,19.06,17.86,0.013889,0.013889,1,1.0273,7.2,2,5.5931,0.960000,29.360000
2,2,"LINESTRING (6365647.387 6173486.807, 6365727.4...",97.9,17.86,16.75,0.011338,0.011338,2,5.4758,46.0,15,11.0689,1.087778,48.175556
3,3,"LINESTRING (6360984.651 6169800.629, 6360993.1...",14.0,26.00,25.96,0.002857,0.003000,3,2.1891,27.7,4,2.1891,0.155556,27.700000
4,4,"LINESTRING (6360993.137 6169789.552, 6361069.9...",122.6,25.96,25.61,0.002855,0.003000,4,2.0797,27.5,17,4.2688,1.362222,29.062222
5,5,"LINESTRING (6361025.835 6169509.217, 6361103.8...",128.4,25.34,25.00,0.002648,0.003000,6,2.8899,30.8,5,2.8899,1.426667,30.800000
6,6,"LINESTRING (6361103.855 6169407.223, 6361219.7...",145.9,25.00,25.00,0.000000,0.003000,5,1.4852,15.5,18,4.3751,1.621111,32.421111
7,7,"LINESTRING (6364547.888 6164990.516, 6364657.3...",149.4,8.24,7.11,0.007564,0.007564,7,1.3926,10.3,8,1.3926,1.660000,10.300000
8,8,"LINESTRING (6364657.304 6164888.792, 6364807.7...",203.5,7.11,6.26,0.004177,0.004177,8,1.3822,10.4,21,2.7748,2.261111,12.661111
9,9,"LINESTRING (6364250.537 6166416.614, 6364163.1...",119.3,21.00,21.00,0.000000,0.003000,9,2.7867,22.6,20,2.7867,1.325556,22.600000


In [35]:
# Predimensionado

C = 0.8
conductos['I_10'] = [int(584.43*conductos['tc'].iloc[i]**-.611) for i in range(len(conductos))]
conductos['Q_10'] = round(C * conductos['I_10'] * conductos['area_acum'] / 360, 2)
conductos['I_5'] = [int(514.81*conductos['tc'].iloc[i]**-.611) for i in range(len(conductos))]
conductos['Q_5'] = round(C * conductos['I_5'] * conductos['area_acum'] / 360, 2)
conductos['I_2'] = [int(435.3*conductos['tc'].iloc[i]**-.611) for i in range(len(conductos))]
conductos['Q_2'] = round(C * conductos['I_2'] * conductos['area_acum'] / 360, 2)

n = 0.012
conductos['D_10'] = round((conductos['Q_10']*n/conductos['i_conducto']**0.5*4**(5/3)/np.pi)**(3/8),2)
conductos['D_5'] = round((conductos['Q_5']*n/conductos['i_conducto']**0.5*4**(5/3)/np.pi)**(3/8),2)
conductos['D_2'] = round((conductos['Q_2']*n/conductos['i_conducto']**0.5*4**(5/3)/np.pi)**(3/8),2)
conductos['D_prov'] = [round(conductos['D_10'].iloc[i],1)*1000 if round(conductos['D_10'].iloc[i],1)*1000>=400 else 400 for i in range(len(conductos))]
conductos.head()

,id_conducto,geometry,longitud,CTNi,CTNf,i,i_conducto,br_inicial,area_br,tv_tc,...,I_10,Q_10,I_5,Q_5,I_2,Q_2,D_10,D_5,D_2,D_prov
0,0,"LINESTRING (6365495.474 6173381.331, 6365576.2...",98.6,19.68,19.06,0.006288,0.006288,0,4.5658,28.4,...,75,0.76,66,0.67,56,0.57,0.69,0.66,0.62,700.0
1,1,"LINESTRING (6365576.298 6173437.724, 6365647.3...",86.4,19.06,17.86,0.013889,0.013889,1,1.0273,7.2,...,74,0.92,65,0.81,55,0.68,0.64,0.61,0.57,600.0
2,2,"LINESTRING (6365647.387 6173486.807, 6365727.4...",97.9,17.86,16.75,0.011338,0.011338,2,5.4758,46.0,...,54,1.33,48,1.18,40,0.98,0.76,0.73,0.68,800.0
3,3,"LINESTRING (6360984.651 6169800.629, 6360993.1...",14.0,26.00,25.96,0.002857,0.003000,3,2.1891,27.7,...,76,0.37,67,0.33,57,0.28,0.60,0.58,0.54,600.0
4,4,"LINESTRING (6360993.137 6169789.552, 6361069.9...",122.6,25.96,25.61,0.002855,0.003000,4,2.0797,27.5,...,74,0.70,65,0.62,55,0.52,0.77,0.73,0.69,800.0


In [36]:
# Matriz de diámetros

mat_apo_d = mat_apo.copy()
for i in range(len(conductos)):
    for j in range(len(conductos)):
        mat_apo_d[i][j] = mat_apo[i][j]*conductos['D_prov'][j]
mat_apo_d.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,700.0,700.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,600.0,600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,600.0,600.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Corrección de diámetros

conductos['D'] = [max([mat_apo_d[i][j] for j in range(len(conductos))]) for i in range(len(conductos))]
conductos.head()

,id_conducto,geometry,longitud,CTNi,CTNf,i,i_conducto,br_inicial,area_br,tv_tc,...,Q_10,I_5,Q_5,I_2,Q_2,D_10,D_5,D_2,D_prov,D
0,0,"LINESTRING (6365495.474 6173381.331, 6365576.2...",98.6,19.68,19.06,0.006288,0.006288,0,4.5658,28.4,...,0.76,66,0.67,56,0.57,0.69,0.66,0.62,700.0,700.0
1,1,"LINESTRING (6365576.298 6173437.724, 6365647.3...",86.4,19.06,17.86,0.013889,0.013889,1,1.0273,7.2,...,0.92,65,0.81,55,0.68,0.64,0.61,0.57,600.0,700.0
2,2,"LINESTRING (6365647.387 6173486.807, 6365727.4...",97.9,17.86,16.75,0.011338,0.011338,2,5.4758,46.0,...,1.33,48,1.18,40,0.98,0.76,0.73,0.68,800.0,800.0
3,3,"LINESTRING (6360984.651 6169800.629, 6360993.1...",14.0,26.00,25.96,0.002857,0.003000,3,2.1891,27.7,...,0.37,67,0.33,57,0.28,0.60,0.58,0.54,600.0,600.0
4,4,"LINESTRING (6360993.137 6169789.552, 6361069.9...",122.6,25.96,25.61,0.002855,0.003000,4,2.0797,27.5,...,0.70,65,0.62,55,0.52,0.77,0.73,0.69,800.0,800.0


In [38]:
# Importar nombre de obras

nombre_objeto_obras = r'C:\Users\Juan Igancio\Desktop\Pluviales\O2023_GIS\SHP\OBRAS.shp'
obras = geopandas.read_file(nombre_objeto_obras)
obras.to_crs(crs = epsg, inplace = True)
obras.head()    

,OBRA,geometry
0,Benito Juárez entre Beiró y Varela,"POLYGON ((6360940.598 6169906.467, 6361077.097..."
1,"Allende entre Simbrón y Tinogasta, Tinogasta e...","POLYGON ((6361128.137 6169498.569, 6361249.040..."
2,Av. De Los Incas entre Estomba y Av. Forest,"POLYGON ((6365747.345 6173552.334, 6365632.089..."
3,Laguna entre Av. Alberdi y Bonifacio,"POLYGON ((6364162.956 6166515.608, 6364303.237..."
4,Moreto entre Monte y Av. Eva Perón,"POLYGON ((6364829.779 6164730.521, 6364618.403..."


In [39]:
# Bautizando shapes

cuencas = cuencas.sjoin(obras, how='left')
cuencas.drop(['index_right'], axis=1, inplace=True)
th = th.sjoin(obras, how='left')
th.drop(['index_right'], axis=1, inplace=True)
sumideros_m = sumideros_m.sjoin(obras, how='left')
sumideros_m.drop(['index_right'], axis=1, inplace=True)
sumideros_m
nexos_m = nexos_m.sjoin(obras, how='left')
nexos_m.drop(['index_right'], axis=1, inplace=True)
br = br.sjoin(obras, how='left')
br.drop(['index_right'], axis=1, inplace=True)
conductos = conductos.sjoin(obras, how='left')
conductos.drop(['index_right'], axis=1, inplace=True)

In [40]:
# Cuencas salida
cuencas.to_file(nombre_objeto_cuencas_s)

# Trayectorias hidráulicas salida
th.to_file(nombre_objeto_th_s)

# Sumideros salida
sumideros_m.to_file(nombre_objeto_sumideros_s)

# Nexos salida
nexos_m.to_file(nombre_objeto_nexos_s)

# BR salida
br.to_file(nombre_objeto_br_s)

# Conductos originales salida
conductos.to_file(nombre_objeto_conductos_s)

C:\Users\Juan Igancio\AppData\Local\Temp\ipykernel_7580\4256212073.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sumideros_m.to_file(nombre_objeto_sumideros_s)
C:\Users\Juan Igancio\AppData\Local\Temp\ipykernel_7580\4256212073.py:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nexos_m.to_file(nombre_objeto_nexos_s)
C:\Users\Juan Igancio\AppData\Local\Temp\ipykernel_7580\4256212073.py:17: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  conductos.to_file(nombre_objeto_conductos_s)
